In [25]:
from bs4 import BeautifulSoup

In [26]:
html_doc = open('List_of_postal_codes_of_Canada__M.html',encoding='utf-8',errors='ignore')

In [27]:
soup=BeautifulSoup(html_doc,'html.parser')

In [28]:
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [29]:
import pandas as pd

In [30]:
dfs = pd.read_html(str(soup.table))

In [31]:
df=dfs[0]

### 1. The dataframe consist of three columns: PostalCode, Borough, and Neighborhood as a result of web scraping

In [32]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 2. In next cell we drop rows where Borough is Not assigned

In [33]:
drop_list=[]

for i in df.index:
    if df.loc[i,'Borough'] == 'Not assigned':
        drop_list.append(i)
        
df.drop(drop_list,axis=0,inplace=True)

In [34]:
df.shape

(210, 3)

### 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [46]:
for i in df.index:
    if df.loc[i,'Neighbourhood'] == 'Not assigned':
        print('Found')
        df.loc[i,'Neighbourhood'] = df.loc[i,'Borough']

In [47]:
df['Postcode'].unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [48]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### 4. More than one neighborhood can exist in one postal code area. In the next cell we club them.

In [38]:
df_new = pd.DataFrame()
df_new['Postcode'] = df['Postcode'].unique()
df_new['Borough'] = pd.np.nan
df_new['Neighbourhood'] = pd.np.nan

for index,val in enumerate(df['Postcode'].unique()):
    temp_list=[]
    for j in df.index:
        if df.loc[j,'Postcode']==val:
            temp_list.append(df.loc[j,'Neighbourhood'])
    #print(i,temp_list)
    df_new.loc[index,'Neighbourhood'] = ', '.join(temp_list)
    df_new.loc[index,'Borough'] = df.loc[j,'Borough']
    

### First Part of Assignment is completed and output is seen in next cell

In [39]:
df_new.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,Etobicoke,Parkwoods
1,M4A,Etobicoke,Victoria Village
2,M5A,Etobicoke,Harbourfront
3,M6A,Etobicoke,"Lawrence Heights, Lawrence Manor"
4,M7A,Etobicoke,Queen's Park
5,M9A,Etobicoke,Queen's Park
6,M1B,Etobicoke,"Rouge, Malvern"
7,M3B,Etobicoke,Don Mills North
8,M4B,Etobicoke,"Woodbine Gardens, Parkview Hill"
9,M5B,Etobicoke,"Ryerson, Garden District"


### Read Location information from csv file

In [40]:
df2=pd.read_csv('Geospatial_Coordinates.csv')

In [41]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
df_new['Latitude'],df_new['Longitude']=pd.np.nan,pd.np.nan

In [44]:
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,Etobicoke,Parkwoods,NaN,NaN
1,M4A,Etobicoke,Victoria Village,NaN,NaN
2,M5A,Etobicoke,Harbourfront,NaN,NaN
3,M6A,Etobicoke,"Lawrence Heights, Lawrence Manor",NaN,NaN
4,M7A,Etobicoke,Queen's Park,NaN,NaN


In [51]:
for i in df2['Postal Code']:
    for j in df_new['Postcode']:
        if i==j:
            df_new['Latitude'] = df2['Latitude']
            df_new['Longitude'] = df2['Longitude']

### Second Part of Assignment is complete. Result in next cell.

In [52]:
df_new.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,Etobicoke,Parkwoods,43.806686,-79.194353
1,M4A,Etobicoke,Victoria Village,43.784535,-79.160497
2,M5A,Etobicoke,Harbourfront,43.763573,-79.188711
3,M6A,Etobicoke,"Lawrence Heights, Lawrence Manor",43.770992,-79.216917
4,M7A,Etobicoke,Queen's Park,43.773136,-79.239476
5,M9A,Etobicoke,Queen's Park,43.744734,-79.239476
6,M1B,Etobicoke,"Rouge, Malvern",43.727929,-79.262029
7,M3B,Etobicoke,Don Mills North,43.711112,-79.284577
8,M4B,Etobicoke,"Woodbine Gardens, Parkview Hill",43.716316,-79.239476
9,M5B,Etobicoke,"Ryerson, Garden District",43.692657,-79.264848
